In [ ]:
import polars as pl

from constants import (
    FE_SRC_FOLDER,
    ID_COL,
    NOT_USE_COL,
    RANK_COL,
    RAW_DATA,
    TOOL_COL,
    TOOL_COUNTS_DATA,
    TOOLS_COL_PREFIX,
    TOOLS_MAP,
    TOOLS_OTHER_COL,
    TOTAL_COUNT_COL,
    USE_COL,
)

In [ ]:
pl.__version__

In [ ]:
df = pl.scan_csv(RAW_DATA)

In [ ]:
# df.collect().shape
# type(df)
# df.fetch(5)
df.fetch(1)
# df.columns

In [ ]:
q_tools = df.select([pl.col(ID_COL), pl.col(f"^{TOOLS_COL_PREFIX}.*$")]).drop(
    TOOLS_OTHER_COL
)

In [ ]:
# tools_cols = [
#     col
#     for col in df.columns
#     if col.startswith(TOOLS_COL_PREFIX) and col != f"{TOOLS_COL_PREFIX}Other__"
# ]
# tools_cols = [
#     pl.col(col)
#     for col in df.columns
#     if (col.startswith(TOOLS_COL_PREFIX) and col != f"{TOOLS_COL_PREFIX}Other__")
#     or col == "chronID"
# ]
# tools_cols

# q_tools = df.select(tools_cols)

In [ ]:
print(q_tools.describe_plan())

In [ ]:
print(q_tools.describe_optimized_plan())

In [ ]:
# df_tools = q_tools.collect(no_optimization=False)
# df_tools = q_tools.collect(no_optimization=True)

# More info:
# - https://github.com/pola-rs/polars/issues/1659#issuecomment-956499225

# df_tools = q_tools.collect(projection_pushdown=True)
df_tools = q_tools.collect(projection_pushdown=False)

In [ ]:
# type(df_tools)
df_tools.shape

In [ ]:
df_tools[ID_COL].n_unique()

In [ ]:
df_tools.head()

In [ ]:
df_tools.tail()

In [ ]:
df_tools[TOOLS_COL_PREFIX].value_counts()

In [ ]:
df_tools[f"{TOOLS_COL_PREFIX}ArcGIS"].value_counts()

In [ ]:
df_tools[f"{TOOLS_COL_PREFIX}D3"].value_counts()

In [ ]:
df_tools.null_count()

In [ ]:
predicate = pl.all().exclude([ID_COL, TOOLS_COL_PREFIX]).str.contains("^$")

# q_count_tools = df_tools.lazy().select(
#     [
#         predicate.sum().suffix("_notuse"),
#         predicate.is_not().sum().suffix("_use"),
#     ]
# )

# q_count_tools.collect().transpose(
#     include_header=True, header_name="tool", column_names=["count"]
# )

q_use_tools = df_tools.lazy().select(predicate.is_not().sum())
# q_use_tools.collect()

q_not_use_tools = df_tools.lazy().select(predicate.sum())
# q_not_use_tools.collect()

In [ ]:
df_use_tools = q_use_tools.collect().transpose(
    include_header=True, header_name=TOOL_COL, column_names=[USE_COL]
)

df_not_use_tools = q_not_use_tools.collect().transpose(
    include_header=True, header_name=TOOL_COL, column_names=[NOT_USE_COL]
)

# df_use_tools.head()
# df_not_use_tools.head()

In [ ]:
# ?pl.Expr.rank

In [ ]:
df_count_tools = df_use_tools.join(df_not_use_tools, on=TOOL_COL)

df_counts_tools = df_count_tools.with_columns(
    [
        pl.col(TOOL_COL).str.slice(start=len(TOOLS_COL_PREFIX)),
        (pl.col(USE_COL) + pl.col(NOT_USE_COL)).alias(TOTAL_COUNT_COL),
        pl.col(USE_COL).rank(method="ordinal", reverse=True).alias(RANK_COL),
    ]
)

In [ ]:
df_counts_tools.head()

In [ ]:
df_counts_tools[RANK_COL].value_counts().sort(by="counts").head()

In [ ]:
df_counts_tools[TOOL_COL].to_list()

In [ ]:
# Adapted from: https://pola-rs.github.io/polars-book/user-guide/howcani/apply/udfs.html#apply-custom-functions
def rename_tools(tool: str) -> str:
    return TOOLS_MAP.get(tool, tool)


df_counts_tools = df_counts_tools.with_column(pl.col(TOOL_COL).apply(rename_tools))

In [ ]:
df_counts_tools.head()

In [ ]:
df_counts_tools.tail()

In [ ]:
df_counts_tools[TOTAL_COUNT_COL].value_counts()

In [ ]:
df_counts_tools.to_csv(f"{TOOL_COUNTS_DATA}.csv")

In [ ]:
FE_SRC_FOLDER.resolve()

In [ ]:
# print(df_counts_tools.to_json(pretty=True))

# indent = 0
indent = 2
df_counts_tools.to_pandas().to_json(
    FE_SRC_FOLDER / f"{TOOL_COUNTS_DATA}.json",
    orient="records",
    force_ascii=False,
    indent=indent,
)

---